NL2Bash using Transformer

# Steps including:

1.loading dataset


3.Defining Transformer model

4.Training Loop

5.Saving model

6.Testing Model






# Loading Dataset


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("/content/dataset.csv")

In [10]:
df.head()

,natural_language,terminal_command
0,List all files in the current directory,ls
1,Show me all the files in this directory,ls
2,Can you display the files in the current folder?,ls
3,Please list everything in this directory,ls
4,Show all files in the current directory,ls


In [12]:
natural_lang=df["natural_language"]
term_cmd=df["terminal_command"]

In [15]:
print(natural_lang[100])
print(term_cmd[100])

Remove the file 'file1.txt'
rm file1.txt


In [19]:
natural_lang.tolist()
term_cmd.tolist()

['ls',
 'ls',
 'ls',
 'ls',
 'ls',
 'ls',
 'ls -a',
 'ls -a',
 'ls -a',
 'ls -a',
 'ls -a',
 'ls -l',
 'ls -l',
 'ls -l',
 'ls -l',
 'ls',
 'ls -l',
 'man ls',
 'cd Documents',
 'cd Documents',
 'cd newDir',
 'cd newDir',
 'cd newDir',
 'cd Documents',
 'cd /home/user/docs',
 'cd /home/user/docs',
 'cd /home/user/docs',
 'cd /home/user/docs',
 'cd /var/logs',
 'cd /var/logs',
 'cd /var/logs',
 'cd /var/logs',
 'cd Documents',
 'cd Documents',
 'cd Documents',
 'cd ~/Documents',
 'cd ~/Documents',
 'cd ..',
 'cd ..',
 'cd ..',
 'cd ..',
 'cd ..',
 'cd ..',
 'cd ..',
 'cd ..',
 'cd ..',
 'cd newDir',
 'cd hello',
 'cd seema',
 'cd Gfg',
 'cd Gfg',
 'cd path',
 'cd seema',
 'cd /home/user/docs',
 'cd /var/logs',
 'cd /',
 'cd /',
 'cd /',
 'cd /',
 'cd ~',
 'cd ~',
 'cd ~',
 'cd ~',
 'exit',
 'exit',
 'exit',
 'exit',
 'exit',
 'exit',
 'exit',
 'exit',
 'exit',
 'exit',
 'exit',
 'exit',
 'exit',
 'exit',
 'exit',
 'kill 12345',
 'kill 12345',
 'kill 12345',
 'kill 12345',
 'kill 12345',

In [18]:
natural_lang[1]

'Show me all the files in this directory'

# Text Preprocessing

# intializing TOkenizer

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
text_tokenizer=Tokenizer()

In [25]:
cmd_tokenizer=Tokenizer()

In [26]:
text_tokenizer.fit_on_texts(natural_lang)
cmd_tokenizer.fit_on_texts(term_cmd)

# Converting sequences

In [93]:
term_cmd[0]

'ls'

In [94]:
text_sequences=text_tokenizer.texts_to_sequences(natural_lang)
cmd_sequence=cmd_tokenizer.texts_to_sequences(term_cmd)


In [95]:
text_sequences[0]

[38, 7, 6, 4, 1, 12, 8]

In [96]:
text_tokenizer.sequences_to_texts([[38, 7, 6, 4, 1, 12, 8]])


['list all files in the current directory']

In [97]:
len(text_sequences)

642

In [98]:
z=cmd_sequence[0]

In [99]:
cmd_tokenizer.sequences_to_texts([z])


['ls']

In [100]:
max_textLen=max(len(text) for text in text_sequences)
max_cmdLen=max(len(cmd) for cmd in cmd_sequence)

In [101]:
max_textLen ,max_cmdLen

(42, 27)

# padding the sentences

In [102]:
text_sequences=pad_sequences(text_sequences,padding="post",maxlen=max_textLen)
cmd_sequence=pad_sequences(cmd_sequence,padding="post",maxlen=max_cmdLen)

In [103]:
x=text_sequences[0]

In [104]:
text_tokenizer.sequences_to_texts([x])


['list all files in the current directory']

In [105]:
x=cmd_sequence[0]

In [106]:
cmd_tokenizer.sequences_to_texts([x])

['ls']

###### BREAK ######

So for now  we have set up our Dataset ,

as in tokenized the sentences
->

show all files =[1,2,3,4,] etc some number

same for cmd

Our Next task will be to implement the architecture of transformer ,small enough to work on google collab, with fine accuracy

# ENCODER PART

We know its parts being :



1. Positional Embedding(after input embedding)
    embedding with context to prev n all
2. Skip connection
3. Multi Head Attention
4. Add & Normalization
5. Feed Forward
6. Add & norm again.
7. passing the output to mha



# POSITIONAL EMBEDDING

 Here th different parts will include 🇰

1.Input Embedding
2. Positional encoding the main part

In [107]:
import tensorflow as tf


In [109]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self,vocab_size,emb_dim):
    super().__init__()
    self.emb_dim=emb_dim
    self.vocab_size=vocab_size
    self.embedding=tf.keras.layers.Embedding(vocab_size,emb_dim,mask_zero=True)
    self.positional_encoding=positional_encoding(length=1000,depth=emb_dim)

  #defining not using padding
  def compute_mask(self,*args,**kwargs):
    return self.embedding.compute_mask(*args,**kwargs)
  def call(self,input_sq):
    length=tf.shape(input_sq)[1] # num of tokens
    input_sq=self.embedding(input_sq)
    #scaled dot product attention
    input_sq*=tf.math.sqrt(tf.cast(self.emb_dim,tf.float32))
    input_sq=input_sq+self.positional_encoding[tf.newaxis,:length,:]
    return input_sq

